**Import Library**

In [57]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from wordcloud import WordCloud

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [58]:
# Membaca dataset ulasan
df = pd.read_csv('ulasan_spotify_playstore.csv')

# Tampilkan 5 baris pertama untuk melihat struktur data
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,8809704b-d271-41e6-8ee1-87ae681b282e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,good boy,5,0,9.0.26.632,2025-03-24 06:55:33,NaN,NaN,9.0.26.632
1,1b05114f-dcf0-46e8-b917-b30e767c613a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,boleh juga sih,4,0,9.0.28.630,2025-03-24 06:53:33,NaN,NaN,9.0.28.630
2,358a0375-b39d-4bfb-b169-0932e327731f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,simple,5,0,9.0.22.543,2025-03-24 06:46:52,NaN,NaN,9.0.22.543
3,eb295400-84bf-46b3-900e-da63bfe1670e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,biar di baca sama owner nya mohon kok setelah ...,1,0,9.0.28.630,2025-03-24 06:41:40,NaN,NaN,9.0.28.630
4,93bc08c6-271b-44ea-92a4-030054b1a713,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,ada iklan nya buat yang ga premium,4,0,8.9.16.593,2025-03-24 06:33:46,NaN,NaN,8.9.16.593


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   reviewId              10000 non-null  object 
 1   userName              10000 non-null  object 
 2   userImage             10000 non-null  object 
 3   content               10000 non-null  object 
 4   score                 10000 non-null  int64  
 5   thumbsUpCount         10000 non-null  int64  
 6   reviewCreatedVersion  7233 non-null   object 
 7   at                    10000 non-null  object 
 8   replyContent          0 non-null      float64
 9   repliedAt             0 non-null      float64
 10  appVersion            7233 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 859.5+ KB


In [60]:
# Tidak menghapus kolom yang relevan, tetap mempertahankan kolom dalam dataset
df_cleaned = df.copy()

# Menangani nilai yang hilang di kolom 'content'
df_cleaned = df_cleaned.dropna(subset=['content'])

# Menampilkan data yang sudah dibersihkan tanpa menghapus kolom yang relevan
df_cleaned.head()


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,8809704b-d271-41e6-8ee1-87ae681b282e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,good boy,5,0,9.0.26.632,2025-03-24 06:55:33,NaN,NaN,9.0.26.632
1,1b05114f-dcf0-46e8-b917-b30e767c613a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,boleh juga sih,4,0,9.0.28.630,2025-03-24 06:53:33,NaN,NaN,9.0.28.630
2,358a0375-b39d-4bfb-b169-0932e327731f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,simple,5,0,9.0.22.543,2025-03-24 06:46:52,NaN,NaN,9.0.22.543
3,eb295400-84bf-46b3-900e-da63bfe1670e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,biar di baca sama owner nya mohon kok setelah ...,1,0,9.0.28.630,2025-03-24 06:41:40,NaN,NaN,9.0.28.630
4,93bc08c6-271b-44ea-92a4-030054b1a713,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,ada iklan nya buat yang ga premium,4,0,8.9.16.593,2025-03-24 06:33:46,NaN,NaN,8.9.16.593


In [61]:
# Mengisi nilai yang hilang pada kolom tertentu
df_cleaned['replyContent'] = df_cleaned['replyContent'].fillna('No Reply')
df_cleaned['repliedAt'] = df_cleaned['repliedAt'].fillna('No Reply')
df_cleaned['reviewCreatedVersion'] = df_cleaned['reviewCreatedVersion'].fillna('No Reply')
# Mengisi nilai yang hilang pada kolom 'appVersion' dengan nilai yang paling sering muncul
most_frequent_version = df_cleaned['appVersion'].mode()[0]
df_cleaned['appVersion'] = df_cleaned['appVersion'].fillna(most_frequent_version)

# Memeriksa kembali apakah ada nilai yang hilang
df_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              10000 non-null  object
 1   userName              10000 non-null  object
 2   userImage             10000 non-null  object
 3   content               10000 non-null  object
 4   score                 10000 non-null  int64 
 5   thumbsUpCount         10000 non-null  int64 
 6   reviewCreatedVersion  10000 non-null  object
 7   at                    10000 non-null  object
 8   replyContent          10000 non-null  object
 9   repliedAt             10000 non-null  object
 10  appVersion            10000 non-null  object
dtypes: int64(2), object(9)
memory usage: 859.5+ KB


In [62]:
# Mengonversi kolom 'at' menjadi datetime
df_cleaned['at'] = pd.to_datetime(df_cleaned['at'])

# Memeriksa tipe data kolom setelah konversi
print(df_cleaned['at'].dtype)

datetime64[ns]


In [63]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              10000 non-null  object        
 1   userName              10000 non-null  object        
 2   userImage             10000 non-null  object        
 3   content               10000 non-null  object        
 4   score                 10000 non-null  int64         
 5   thumbsUpCount         10000 non-null  int64         
 6   reviewCreatedVersion  10000 non-null  object        
 7   at                    10000 non-null  datetime64[ns]
 8   replyContent          10000 non-null  object        
 9   repliedAt             10000 non-null  object        
 10  appVersion            10000 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 859.5+ KB


In [64]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [65]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [66]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
df_cleaned['text_clean'] = df_cleaned['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
df_cleaned['text_casefoldingText'] = df_cleaned['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
df_cleaned['text_slangwords'] = df_cleaned['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
df_cleaned['text_tokenizingText'] = df_cleaned['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
df_cleaned['text_stopword'] = df_cleaned['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
df_cleaned['text_akhir'] = df_cleaned['text_stopword'].apply(toSentence)

In [67]:
df_cleaned

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,8809704b-d271-41e6-8ee1-87ae681b282e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,good boy,5,0,9.0.26.632,2025-03-24 06:55:33,No Reply,No Reply,9.0.26.632,good boy,good boy,good boy,"[good, boy]","[good, boy]",good boy
1,1b05114f-dcf0-46e8-b917-b30e767c613a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,boleh juga sih,4,0,9.0.28.630,2025-03-24 06:53:33,No Reply,No Reply,9.0.28.630,boleh juga sih,boleh juga sih,boleh juga sih,"[boleh, juga, sih]",[],
2,358a0375-b39d-4bfb-b169-0932e327731f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,simple,5,0,9.0.22.543,2025-03-24 06:46:52,No Reply,No Reply,9.0.22.543,simple,simple,simple,[simple],[simple],simple
3,eb295400-84bf-46b3-900e-da63bfe1670e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,biar di baca sama owner nya mohon kok setelah ...,1,0,9.0.28.630,2025-03-24 06:41:40,No Reply,No Reply,9.0.28.630,biar di baca sama owner nya mohon kok setelah ...,biar di baca sama owner nya mohon kok setelah ...,biar di baca sama owner nya mohon kok setelah ...,"[biar, di, baca, sama, owner, nya, mohon, kok,...","[biar, baca, owner, mohon, update, kalo, ganti...",biar baca owner mohon update kalo ganti lagu p...
4,93bc08c6-271b-44ea-92a4-030054b1a713,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,ada iklan nya buat yang ga premium,4,0,8.9.16.593,2025-03-24 06:33:46,No Reply,No Reply,8.9.16.593,ada iklan nya buat yang ga premium,ada iklan nya buat yang ga premium,ada iklan nya buat yang ga premium,"[ada, iklan, nya, buat, yang, ga, premium]","[iklan, premium]",iklan premium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,223c979a-17a7-4cb1-a796-55c179206525,Nurul putri Pratiwi4ww,https://play-lh.googleusercontent.com/a/ACg8oc...,sukakkkk bgttt,5,0,9.0.12.567,2025-01-27 11:46:40,No Reply,No Reply,9.0.12.567,sukakkkk bgttt,sukakkkk bgttt,sukakkkk bgttt,"[sukakkkk, bgttt]","[sukakkkk, bgttt]",sukakkkk bgttt
9996,d4c8ed6f-223c-413f-86b9-f0e53e3b9aac,Putri Lihasa,https://play-lh.googleusercontent.com/a/ACg8oc...,bagus,5,0,9.0.12.567,2025-01-27 11:16:01,No Reply,No Reply,9.0.12.567,bagus,bagus,bagus,[bagus],[bagus],bagus
9997,8423d5fc-d9a6-4c6b-a301-9d3113397a1d,Muni,https://play-lh.googleusercontent.com/a-/ALV-U...,Aplikasinya bagus,5,0,9.0.12.567,2025-01-27 11:08:57,No Reply,No Reply,9.0.12.567,Aplikasinya bagus,aplikasinya bagus,aplikasinya bagus,"[aplikasinya, bagus]","[aplikasinya, bagus]",aplikasinya bagus
9998,53ac6594-44b3-40fe-b394-9732224401ef,Hendra Hidayat,https://play-lh.googleusercontent.com/a/ACg8oc...,Mantap,5,0,9.0.12.567,2025-01-27 10:44:15,No Reply,No Reply,9.0.12.567,Mantap,mantap,mantap,[mantap],[mantap],mantap


In [68]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [69]:
# Fungsi untuk menentukan polaritas sentimen dari teks
def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if word in lexicon_positive:
            score += lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if word in lexicon_negative:
            score += lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    # Menentukan polaritas berdasarkan skor
    if score >= 4:
        polarity = 'positive'  # Positif jika skor 4-5
    elif score == 3:
        polarity = 'neutral'   # Netral jika skor 3
    elif score <= 2:
        polarity = 'negative'  # Negatif jika skor 1-2

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [70]:
results = df_cleaned['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
df_cleaned['polarity_score'] = results[0]
df_cleaned['polarity'] = results[1]
print(df_cleaned['polarity'].value_counts())

polarity
negative    7007
positive    2536
neutral      457
Name: count, dtype: int64


Eksplorasi Label

Logistic Regression + TF-IDF + Pembagian data 80/20

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Pisahkan data menjadi fitur (teks) dan label (sentimen)
X = df_cleaned['text_akhir']
y = df_cleaned['polarity']

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8)
X_tfidf = tfidf.fit_transform(X)

# Membagi data menjadi data latih dan data uji 80/20
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Membuat objek model Logistic Regression
log_reg = LogisticRegression(max_iter=200)

# Melatih model Logistic Regression pada data pelatihan
log_reg.fit(X_train, y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_log_reg = log_reg.predict(X_train)
y_pred_test_log_reg = log_reg.predict(X_test)

# Evaluasi akurasi model Logistic Regression
accuracy_train_log_reg = accuracy_score(y_train, y_pred_train_log_reg)
accuracy_test_log_reg = accuracy_score(y_test, y_pred_test_log_reg)

print('Logistic Regression + TF-IDF - accuracy_train:', accuracy_train_log_reg)
print('Logistic Regression + TF-IDF - accuracy_test:', accuracy_test_log_reg)

Logistic Regression + TF-IDF - accuracy_train: 0.89675
Logistic Regression + TF-IDF - accuracy_test: 0.884


Random Forest + TF-IDF + Pembagian Data 80/20

In [72]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8)
X_tfidf = tfidf.fit_transform(df_cleaned['text_akhir'])
y = df_cleaned['polarity']

# Membagi data menjadi data latih dan data uji 80/20
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Melatih model Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_2.toarray(), y_train_2)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = rf_model.predict(X_train_2.toarray())
y_pred_test_rf = rf_model.predict(X_test_2.toarray())

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train_2)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test_2)

# Menampilkan akurasi
print('Random Forest + TF-IDF - accuracy_train:', accuracy_train_rf)
print('Random Forest + TF-IDF - accuracy_test:', accuracy_test_rf)

Random Forest + TF-IDF - accuracy_train: 0.977125
Random Forest + TF-IDF - accuracy_test: 0.8885


Random Forest + CountVectorizer + Pembagian Data 80/20

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Menggunakan CountVectorizer untuk ekstraksi fitur
count_vectorizer = CountVectorizer(max_features=200, min_df=5, max_df=0.8)
X_count = count_vectorizer.fit_transform(df_cleaned['text_akhir'])

# Target (label sentimen)
y = df_cleaned['polarity']

# Pembagian data menjadi data latih dan data uji 80/20
X_train, X_test, y_train, y_test = train_test_split(X_count, y, test_size=0.2, random_state=42)

# Melatih model Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Prediksi sentimen pada data latih dan data uji
y_pred_train_rf = rf_model.predict(X_train)
y_pred_test_rf = rf_model.predict(X_test)

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest + CountVectorizer - accuracy_train:', accuracy_train_rf)
print('Random Forest + CountVectorizer - accuracy_test:', accuracy_test_rf)

Random Forest + CountVectorizer - accuracy_train: 0.978125
Random Forest + CountVectorizer - accuracy_test: 0.8785


Interface

In [78]:
# Menggunakan teks dari kolom 'text_akhir' pada dataset df_cleaned untuk inference
test_texts = df_cleaned['text_akhir'].head(5).tolist()

# Melakukan inference (testing) untuk teks dari dataset
predicted_sentiments = [sentiment_analyzer.analyze(text) for text in test_texts]

# Menampilkan hasil prediksi
for text, sentiment in zip(test_texts, predicted_sentiments):
    print(f"Review: {text}")
    print(f"Predicted Sentiment: {sentiment}")
    print('-' * 80)


Review: good boy
Predicted Sentiment: negative
--------------------------------------------------------------------------------
Review: 
Predicted Sentiment: negative
--------------------------------------------------------------------------------
Review: simple
Predicted Sentiment: negative
--------------------------------------------------------------------------------
Review: biar baca owner mohon update kalo ganti lagu premium padahl enak banget dengerin lagu terbatas kayak paksa beli premium sekian critik
Predicted Sentiment: negative
--------------------------------------------------------------------------------
Review: iklan premium
Predicted Sentiment: negative
--------------------------------------------------------------------------------
